# होटल बुकिङमा प्राथमिकता सदस्य मिडलवेयर

यो नोटबुकले **फङ्सन-आधारित मिडलवेयर** लाई Microsoft Agent Framework प्रयोग गरेर प्रदर्शन गर्दछ। हामी सशर्त कार्यप्रवाह उदाहरणलाई विस्तार गर्दै प्राथमिकता सदस्यहरूलाई विशेष सुविधा दिने मिडलवेयर तह थप्छौं।

## तपाईंले के सिक्नुहुनेछ:
1. **फङ्सन-आधारित मिडलवेयर**: फङ्सनको नतिजा रोक्ने र परिवर्तन गर्ने
2. **सन्दर्भ पहुँच**: `context.result` लाई कार्यान्वयनपछि पढ्ने र परिवर्तन गर्ने
3. **व्यापारिक तर्क कार्यान्वयन**: प्राथमिकता सदस्य सुविधाहरू
4. **नतिजा अधिलेखन**: प्रयोगकर्ताको स्थिति अनुसार फङ्सनको नतिजा परिवर्तन गर्ने
5. **उही कार्यप्रवाह, फरक नतिजा**: मिडलवेयरद्वारा व्यवहार परिवर्तन

## मिडलवेयरसहित कार्यप्रवाहको संरचना:

```
User Input: "I want to book a hotel in Paris"
                    ↓
        [availability_agent]
        - Calls hotel_booking tool
        - 🌟 priority_check middleware intercepts
        - Checks user membership status
        - IF priority + no rooms → Override to available!
        - Returns BookingCheckResult
                    ↓
        Conditional Routing
           /                    \
    [has_availability]    [no_availability]
          ↓                      ↓
    [booking_agent]        [alternative_agent]
    (Priority override!)   (Regular users)
          ↓                      ↓
       [display_result executor]
```

## सशर्त कार्यप्रवाहबाट मुख्य भिन्नता:

**मिडलवेयर बिना** (14-conditional-workflow.ipynb):
- पेरिसमा कोठा छैन → alternative_agent मा रुट गरिन्छ

**मिडलवेयरसहित** (यो नोटबुक):
- नियमित प्रयोगकर्ता + पेरिस → कोठा छैन → alternative_agent मा रुट गरिन्छ
- प्राथमिकता प्रयोगकर्ता + पेरिस → 🌟 मिडलवेयर अधिलेखन! → उपलब्ध → booking_agent मा रुट गरिन्छ

## आवश्यकताहरू:
- Microsoft Agent Framework स्थापना गरिएको
- सशर्त कार्यप्रवाहको समझ (14-conditional-workflow.ipynb हेर्नुहोस्)
- GitHub टोकन वा OpenAI API कुञ्जी
- मिडलवेयर ढाँचाको आधारभूत समझ


In [2]:
import asyncio
import json
import os
from collections.abc import Awaitable, Callable
from typing import Annotated, Any, Never

from agent_framework import (
    AgentExecutor,
    AgentExecutorRequest,
    AgentExecutorResponse,
    ChatMessage,
    FunctionInvocationContext,
    Role,
    WorkflowBuilder,
    WorkflowContext,
    ai_function,
    executor,
)

# 🤖 GitHub Models or OpenAI client integration
from agent_framework.openai import OpenAIChatClient
from dotenv import load_dotenv
from IPython.display import HTML, display
from pydantic import BaseModel

print("✅ All imports successful!")

✅ All imports successful!


## चरण १: संरचित आउटपुटहरूको लागि Pydantic मोडेलहरू परिभाषित गर्नुहोस्

यी मोडेलहरूले एजेन्टहरूले फिर्ता गर्ने **schema** परिभाषित गर्छन्। हामीले `priority_override` क्षेत्र थपेका छौं जसले मिडलवेयरले उपलब्धता परिणाम परिवर्तन गर्दा ट्र्याक गर्छ।


In [3]:
class BookingCheckResult(BaseModel):
    """Result from checking hotel availability at a destination."""

    destination: str
    has_availability: bool
    message: str
    priority_override: bool = False  # 🆕 NEW! Tracks if middleware overrode the result


class AlternativeResult(BaseModel):
    """Suggested alternative destination when no rooms available."""

    alternative_destination: str
    reason: str


class BookingConfirmation(BaseModel):
    """Booking suggestion when rooms are available."""

    destination: str
    action: str
    message: str


print("✅ Pydantic models defined:")
print("   - BookingCheckResult (availability check with priority_override)")
print("   - AlternativeResult (alternative suggestion)")
print("   - BookingConfirmation (booking confirmation)")

✅ Pydantic models defined:
   - BookingCheckResult (availability check with priority_override)
   - AlternativeResult (alternative suggestion)
   - BookingConfirmation (booking confirmation)


## चरण २: प्राथमिक सदस्यहरूको डाटाबेस परिभाषित गर्नुहोस्

यस डेमोका लागि, हामी प्राथमिक सदस्यता डाटाबेसलाई अनुकरण गर्नेछौं। उत्पादनमा, यसले वास्तविक डाटाबेस वा API सोधपुछ गर्नेछ।

**प्राथमिक सदस्यहरू:**
- `alice@example.com` - VIP सदस्य
- `bob@example.com` - प्रिमियम सदस्य  
- `priority_user` - परीक्षण खाता


In [4]:
# Simulated priority members database
PRIORITY_MEMBERS = {
    "alice@example.com",
    "bob@example.com",
    "priority_user",
}

# Global variable to track current user (in real app, use proper session management)
current_user_id = "regular_user"  # Default: regular user


def set_user(user_id: str):
    """Set the current user for the session."""
    global current_user_id
    current_user_id = user_id
    is_priority = user_id in PRIORITY_MEMBERS
    status = "🌟 PRIORITY MEMBER" if is_priority else "👤 Regular User"

    display(
        HTML(f"""
        <div style='padding: 15px; background: {"linear-gradient(135deg, #FFD700 0%, #FFA500 100%)" if is_priority else "#e3f2fd"}; 
                    border-left: 4px solid {"#FF6B35" if is_priority else "#2196f3"}; border-radius: 4px; margin: 10px 0;'>
            <strong>👤 Current User Set:</strong> {user_id}<br>
            <strong>Status:</strong> {status}
        </div>
    """)
    )


print("✅ Priority members database created")
print(f"   Priority members: {len(PRIORITY_MEMBERS)} users")

✅ Priority members database created
   Priority members: 3 users


## चरण ३: होटल बुकिङ उपकरण बनाउनुहोस्

सर्त अनुसारको कार्यप्रवाह जस्तै, तर अब यो मध्यवर्ती सफ्टवेयरद्वारा रोकिनेछ!


In [5]:
@ai_function(description="Check hotel room availability for a destination city")
def hotel_booking(destination: Annotated[str, "The destination city to check for hotel rooms"]) -> str:
    """
    Simulates checking hotel room availability.
    
    Returns JSON string with availability status.
    """
    display(
        HTML(f"""
        <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
            <strong>🔍 Tool Invoked:</strong> hotel_booking("{destination}")
        </div>
    """)
    )

    # Simulate availability check
    cities_with_rooms = ["stockholm", "seattle", "tokyo", "london", "amsterdam"]
    has_rooms = destination.lower() in cities_with_rooms

    result = {"has_availability": has_rooms, "destination": destination}

    return json.dumps(result)


print("✅ hotel_booking tool created with @ai_function decorator")

✅ hotel_booking tool created with @ai_function decorator


## चरण ४: 🌟 प्राथमिकता जाँच मिडलवेयर बनाउनुहोस् (मुख्य विशेषता!)

यो नोटबुकको **मुख्य कार्यक्षमता** हो। मिडलवेयर:

1. **अवरोध गर्दछ** hotel_booking फङ्सन कललाई
2. `next(context)` कल गरेर फङ्सनलाई सामान्य रूपमा **कार्यान्वयन गर्दछ**
3. `context.result` मा नतिजा **जाँच गर्दछ**
4. यदि प्रयोगकर्ता प्राथमिकता हो र कोठा उपलब्ध छैन भने नतिजा **अधिकारित गर्दछ**
5. **परिमार्जित नतिजा** एजेन्टलाई फिर्ता दिन्छ

**मुख्य ढाँचा:**
```python
async def my_middleware(context, next):
    await next(context)  # Execute function
    # Now context.result contains the function's output
    if some_condition:
        context.result = new_value  # Override!
```


In [6]:
async def priority_check_middleware(
    context: FunctionInvocationContext,
    next: Callable[[FunctionInvocationContext], Awaitable[None]],
) -> None:
    """
    Function middleware that overrides hotel_booking results for priority members.
    
    Workflow:
    1. Let the function execute normally
    2. Check if user is a priority member
    3. If priority + no availability → Override to make rooms available!
    4. Agent will then route to booking path instead of alternative path
    """
    function_name = context.function.name

    display(
        HTML(f"""
        <div style='padding: 12px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
            <strong>🔄 Middleware:</strong> Intercepting {function_name}...
        </div>
    """)
    )

    # Execute the original function
    await next(context)

    # Now inspect and potentially modify the result
    if context.result and function_name == "hotel_booking":
        result_data = json.loads(context.result)
        destination = result_data.get("destination", "")
        has_availability = result_data.get("has_availability", False)

        # Check if user is priority member
        is_priority = current_user_id in PRIORITY_MEMBERS

        # Override logic: Priority member + no availability → Make available!
        if is_priority and not has_availability:
            display(
                HTML(f"""
                <div style='padding: 20px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); 
                            border-radius: 8px; margin: 10px 0; box-shadow: 0 4px 12px rgba(255,165,0,0.4);'>
                    <h3 style='margin: 0 0 10px 0; color: #333;'>🌟 PRIORITY OVERRIDE ACTIVATED! 🌟</h3>
                    <p style='margin: 0; color: #555; line-height: 1.6;'>
                        <strong>User:</strong> {current_user_id}<br>
                        <strong>Status:</strong> VIP Priority Member<br>
                        <strong>Action:</strong> Overriding "No Availability" for {destination}<br>
                        <strong>Result:</strong> ✅ Rooms now available for priority booking!
                    </p>
                </div>
            """)
            )

            # Override the result!
            result_data["has_availability"] = True
            result_data["priority_override"] = True
            context.result = json.dumps(result_data)

        elif not has_availability:
            display(
                HTML(f"""
                <div style='padding: 12px; background: #ffebee; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                    <strong>ℹ️ Middleware:</strong> No priority override (user: {current_user_id})
                </div>
            """)
            )


print("✅ priority_check_middleware created")
print("   - Intercepts hotel_booking function")
print("   - Overrides availability for priority members")

✅ priority_check_middleware created
   - Intercepts hotel_booking function
   - Overrides availability for priority members


## चरण ५: रुटिङका लागि सर्त कार्यहरू परिभाषित गर्नुहोस्

उस्तै सर्त कार्यहरू जस्तै सर्तात्मक कार्यप्रवाह - ती संरचित आउटपुटलाई निरीक्षण गर्छन् र रुटिङ निर्धारण गर्छन्।


In [7]:
def has_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels ARE available (including priority overrides!)."""
    if not isinstance(message, AgentExecutorResponse):
        return True

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        # Check if this was a priority override
        override_indicator = " 🌟" if result.priority_override else ""

        display(
            HTML(f"""
            <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                <strong>✅ Condition Check:</strong> has_availability = <strong>{result.has_availability}</strong> for {result.destination}{override_indicator}
            </div>
        """)
        )

        return result.has_availability
    except Exception as e:
        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                <strong>⚠️  Error:</strong> {str(e)}
            </div>
        """)
        )
        return False


def no_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels are NOT available."""
    if not isinstance(message, AgentExecutorResponse):
        return False

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffecb3; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                <strong>❌ Condition Check:</strong> no_availability for {result.destination}
            </div>
        """)
        )

        return not result.has_availability
    except Exception:
        return False


print("✅ Condition functions defined")

✅ Condition functions defined


## चरण ६: कस्टम डिस्प्ले एक्जिक्युटर बनाउनुहोस्

पहिलेको जस्तै एक्जिक्युटर - अन्तिम वर्कफ्लोको नतिजा देखाउँछ।


In [8]:
@executor(id="display_result")
async def display_result(response: AgentExecutorResponse, ctx: WorkflowContext[Never, str]) -> None:
    """Display the final result as workflow output."""
    display(
        HTML("""
        <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
            <strong>📤 Display Executor:</strong> Yielding workflow output
        </div>
    """)
    )

    await ctx.yield_output(response.agent_run_response.text)


print("✅ display_result executor created")

✅ display_result executor created


## चरण ७: वातावरण चरहरू लोड गर्नुहोस्

LLM क्लाइन्ट (GitHub Models वा OpenAI) कन्फिगर गर्नुहोस्।


In [10]:
# Load environment variables
load_dotenv()

# Check for GitHub Models or OpenAI
chat_client = OpenAIChatClient(base_url=os.environ.get(
    "GITHUB_ENDPOINT"), api_key=os.environ.get("GITHUB_TOKEN"), model_id="gpt-4o")


## चरण ८: मिडलवेयरको साथमा एआई एजेन्टहरू बनाउनुहोस्

**मुख्य भिन्नता:** जब availability_agent बनाइन्छ, हामी `middleware` प्यारामिटर पास गर्छौं!

यसरी हामी priority_check_middleware लाई एजेन्टको फङ्सन कल गर्ने पाइपलाइनमा समावेश गर्छौं।


In [ ]:
# Agent 1: Check availability with tool + middleware
availability_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a hotel booking assistant that checks room availability. "
            "Use the hotel_booking tool to check if rooms are available at the destination. "
            "Return JSON with fields: destination (string), has_availability (bool), message (string), "
            "and priority_override (bool, true if priority member got special access). "
            "The message should summarize the availability status and mention if priority override occurred."
        ),
        tools=[hotel_booking],
        response_format=BookingCheckResult,
        middleware=[priority_check_middleware],  # 🌟 MIDDLEWARE INJECTION!
    ),
    id="availability_agent",
)

# Agent 2: Suggest alternative (when no rooms)
alternative_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful travel assistant. When a user cannot find hotels in their requested city, "
            "suggest an alternative nearby city that has availability. "
            "Return JSON with fields: alternative_destination (string) and reason (string). "
            "Make your suggestion sound appealing and helpful."
        ),
        response_format=AlternativeResult,
    ),
    id="alternative_agent",
)

# Agent 3: Suggest booking (when rooms available)
booking_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a booking assistant. The user has found available hotel rooms. "
            "Encourage them to book by highlighting the destination's appeal. "
            "If priority_override is true in the input, mention that they received priority member access. "
            "Return JSON with fields: destination (string), action (string), and message (string). "
            "The action should be 'book_now' and message should be encouraging."
        ),
        response_format=BookingConfirmation,
    ),
    id="booking_agent",
)

display(
    HTML("""
    <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
        <strong>✅ Created 3 Agents:</strong>
        <ul style='margin: 10px 0 0 0;'>
            <li><strong>availability_agent</strong> - WITH priority_check_middleware 🌟</li>
            <li><strong>alternative_agent</strong> - Suggests alternative cities</li>
            <li><strong>booking_agent</strong> - Encourages booking</li>
        </ul>
    </div>
""")
)

## चरण ९: वर्कफ्लो निर्माण गर्नुहोस्

पहिले जस्तै वर्कफ्लो संरचना - उपलब्धतामा आधारित सशर्त रुटिङ।


In [12]:
# Build the workflow with conditional routing
workflow = (
    WorkflowBuilder()
    .set_start_executor(availability_agent)
    # NO AVAILABILITY PATH
    .add_edge(availability_agent, alternative_agent, condition=no_availability_condition)
    .add_edge(alternative_agent, display_result)
    # HAS AVAILABILITY PATH (can be triggered by middleware override!)
    .add_edge(availability_agent, booking_agent, condition=has_availability_condition)
    .add_edge(booking_agent, display_result)
    .build()
)

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; border-radius: 8px; margin: 10px 0;'>
        <h3 style='margin: 0 0 15px 0;'>✅ Workflow Built Successfully!</h3>
        <p style='margin: 0; line-height: 1.6;'>
            <strong>Conditional Routing (Middleware-Aware):</strong><br>
            • If <strong>NO availability</strong> → alternative_agent → display_result<br>
            • If <strong>availability</strong> (or 🌟 <strong>priority override</strong>) → booking_agent → display_result
        </p>
    </div>
""")
)

## चरण १०: परीक्षण केस १ - सामान्य प्रयोगकर्ता पेरिसमा (कुनै ओभरराइड छैन)

सामान्य प्रयोगकर्ताले पेरिस बुक गर्न प्रयास गर्छ → कुनै कोठा उपलब्ध छैन → वैकल्पिक एजेन्टमा रुट हुन्छ।


In [13]:
# Set as regular user
set_user("regular_user")

display(
    HTML("""
    <div style='padding: 20px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #e65100;'>🧪 TEST CASE 1: Regular User + Paris</h3>
        <p style='margin: 0;'><strong>Expected:</strong> No rooms → No middleware override → Alternative suggestion</p>
    </div>
""")
)

# Create request
request_regular = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], should_respond=True
)

# Run workflow
events_regular = await workflow.run(request_regular)
outputs_regular = events_regular.get_outputs()

# Display results
if outputs_regular:
    result_regular = AlternativeResult.model_validate_json(outputs_regular[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: #fff; border: 2px solid #ff9800; border-radius: 12px; margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0; color: #e65100;'>📊 RESULT (Regular User)</h3>
            <div style='background: #fff3e0; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                <p style='margin: 0 0 10px 0;'><strong>Middleware:</strong> No priority override (regular user)</p>
                <p style='margin: 0 0 10px 0;'><strong>Alternative:</strong> 🏨 {result_regular.alternative_destination}</p>
                <p style='margin: 0;'><strong>Reason:</strong> {result_regular.reason}</p>
            </div>
        </div>
    """)
    )

## चरण ११: परीक्षण केस २ - 🌟 प्राथमिकता प्रयोगकर्ता पेरिसमा (ओभरराइड सहित!)

एक प्राथमिकता सदस्यले पेरिस बुक गर्न प्रयास गर्छ → सुरुमा कोठा उपलब्ध छैन → 🌟 मिडलवेयरले ओभरराइड गर्छ! → booking_agent मा रुट हुन्छ

**यो मिडलवेयरको शक्तिको मुख्य प्रदर्शन हो!**


In [14]:
# Set as priority user
set_user("priority_user")

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #333;'>🧪 TEST CASE 2: 🌟 Priority User + Paris</h3>
        <p style='margin: 0; color: #555;'><strong>Expected:</strong> No rooms → 🌟 MIDDLEWARE OVERRIDE → Rooms available → Booking suggestion!</p>
    </div>
""")
)

# Create request
request_priority = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], should_respond=True
)

# Run workflow
events_priority = await workflow.run(request_priority)
outputs_priority = events_priority.get_outputs()

# Display results
if outputs_priority:
    result_priority = BookingConfirmation.model_validate_json(outputs_priority[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 12px; 
                    box-shadow: 0 8px 16px rgba(255,165,0,0.4); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0; color: #333;'>🏆 RESULT (Priority Member) 🌟</h3>
            <div style='background: white; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available (Priority Override!)</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Middleware:</strong> 🌟 OVERRIDE ACTIVATED!</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_priority.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_priority.action}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_priority.message}</p>
                <div style='margin-top: 15px; padding: 15px; background: #fff3cd; border-radius: 6px; border-left: 4px solid #FF6B35;'>
                    <strong>💡 What Just Happened:</strong><br>
                    1. hotel_booking tool returned "no availability"<br>
                    2. priority_check_middleware intercepted the result<br>
                    3. Middleware checked user status: priority_user ✅<br>
                    4. Middleware OVERRODE the result to "available"<br>
                    5. Workflow routed to booking_agent instead of alternative_agent!
                </div>
            </div>
        </div>
    """)
    )

## चरण १२: परीक्षण केस ३ - स्टकहोममा प्राथमिकता प्रयोगकर्ता (पहिले नै उपलब्ध)

प्राथमिकता प्रयोगकर्ता स्टकहोम प्रयास गर्छ → कोठाहरू उपलब्ध छन् → कुनै ओभरराइड आवश्यक छैन → booking_agent मा रुट हुन्छ

यसले देखाउँछ कि मिडलवेयर केवल आवश्यक पर्दा मात्र कार्य गर्छ!


In [15]:
# Priority user is still set from previous test

display(
    HTML("""
    <div style='padding: 20px; background: #e8f5e9; border-left: 4px solid #4caf50; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #1b5e20;'>🧪 TEST CASE 3: Priority User + Stockholm</h3>
        <p style='margin: 0;'><strong>Expected:</strong> Rooms available → No override needed → Booking suggestion</p>
    </div>
""")
)

# Create request
request_stockholm = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Stockholm")], should_respond=True
)

# Run workflow
events_stockholm = await workflow.run(request_stockholm)
outputs_stockholm = events_stockholm.get_outputs()

# Display results
if outputs_stockholm:
    result_stockholm = BookingConfirmation.model_validate_json(outputs_stockholm[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #4caf50 0%, #8bc34a 100%); color: white; border-radius: 12px; 
                    box-shadow: 0 4px 12px rgba(76,175,80,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0;'>🏆 RESULT (Priority User - No Override Needed)</h3>
            <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available (Natural)</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Middleware:</strong> No override needed</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_stockholm.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_stockholm.action}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_stockholm.message}</p>
                <div style='margin-top: 15px; padding: 15px; background: #e8f5e9; border-radius: 6px; border-left: 4px solid #4caf50;'>
                    <strong>💡 Middleware Behavior:</strong><br>
                    • hotel_booking returned "available" naturally<br>
                    • Middleware saw available = true → No override needed<br>
                    • Workflow proceeded normally to booking_agent
                </div>
            </div>
        </div>
    """)
    )

## मुख्य बुँदाहरू र मिडलवेयर अवधारणाहरू

### ✅ तपाईंले सिक्नुभएको:

#### **1. फङ्सन-आधारित मिडलवेयर ढाँचा**

मिडलवेयरले एक साधारण असिन्क्रोनस फङ्सन प्रयोग गरेर फङ्सन कलहरू रोक्छ:

```python
async def my_middleware(
    context: FunctionInvocationContext,
    next: Callable,
) -> None:
    # Before function execution
    print("Intercepting...")
    
    # Execute the function
    await next(context)
    
    # After function execution - inspect result
    if context.result:
        # Modify result if needed
        context.result = modified_value
```

#### **2. कन्टेक्स्ट पहुँच र परिणाम ओभरराइड**

- `context.function` - कल भइरहेको फङ्सनमा पहुँच
- `context.arguments` - फङ्सनको आर्गुमेन्टहरू पढ्नुहोस्
- `context.kwargs` - थप प्यारामिटरहरूमा पहुँच
- `await next(context)` - फङ्सन कार्यान्वयन गर्नुहोस्
- `context.result` - फङ्सनको आउटपुट पढ्नुहोस्/परिमार्जन गर्नुहोस्

#### **3. व्यवसायिक तर्क कार्यान्वयन**

हाम्रो मिडलवेयरले प्राथमिक सदस्य सुविधाहरू कार्यान्वयन गर्दछ:
- **सामान्य प्रयोगकर्ता**: कुनै परिमार्जन छैन, मानक कार्यप्रवाह
- **प्राथमिक प्रयोगकर्ता**: "उपलब्धता छैन" → "उपलब्ध" ओभरराइड
- **सर्तात्मक तर्क**: आवश्यक पर्दा मात्र ओभरराइड गर्दछ

#### **4. एउटै कार्यप्रवाह, फरक परिणामहरू**

मिडलवेयरको शक्ति:
- ✅ कार्यप्रवाह संरचनामा कुनै परिवर्तन छैन
- ✅ टूल फङ्सनमा कुनै परिवर्तन छैन
- ✅ सर्तात्मक रुटिङ तर्कमा कुनै परिवर्तन छैन
- ✅ केवल मिडलवेयर → फरक व्यवहार!

### 🚀 वास्तविक-जीवन अनुप्रयोगहरू:

1. **VIP/प्रिमियम सुविधाहरू**
   - प्रिमियम प्रयोगकर्ताहरूको लागि दर सीमा ओभरराइड गर्नुहोस्
   - स्रोतहरूमा प्राथमिक पहुँच प्रदान गर्नुहोस्
   - प्रिमियम सुविधाहरू गतिशील रूपमा अनलक गर्नुहोस्

2. **A/B परीक्षण**
   - प्रयोगकर्ताहरूलाई फरक कार्यान्वयनमा रुट गर्नुहोस्
   - विशिष्ट प्रयोगकर्ताहरूसँग नयाँ सुविधाहरू परीक्षण गर्नुहोस्
   - सुविधाहरू क्रमिक रूपमा रोलआउट गर्नुहोस्

3. **सुरक्षा र अनुपालन**
   - फङ्सन कलहरू अडिट गर्नुहोस्
   - संवेदनशील अपरेसनहरू ब्लक गर्नुहोस्
   - व्यवसायिक नियमहरू लागू गर्नुहोस्

4. **प्रदर्शन अनुकूलन**
   - विशिष्ट प्रयोगकर्ताहरूको लागि परिणाम क्यास गर्नुहोस्
   - सम्भव हुँदा महँगो अपरेसनहरू स्किप गर्नुहोस्
   - गतिशील स्रोत आवंटन

5. **त्रुटि ह्यान्डलिङ र पुन: प्रयास**
   - त्रुटिहरूलाई सुन्दर रूपमा समात्नुहोस् र ह्यान्डल गर्नुहोस्
   - पुन: प्रयास तर्क कार्यान्वयन गर्नुहोस्
   - वैकल्पिक कार्यान्वयनमा फलब्याक गर्नुहोस्

6. **लगिङ र अनुगमन**
   - फङ्सन कार्यान्वयन समय ट्र्याक गर्नुहोस्
   - प्यारामिटरहरू र परिणामहरू लग गर्नुहोस्
   - प्रयोगका ढाँचाहरू अनुगमन गर्नुहोस्

### 🔑 डेकोरेटरहरूबाट मुख्य भिन्नताहरू:

| विशेषता | डेकोरेटर | मिडलवेयर |
|---------|-----------|------------|
| **स्कोप** | एकल फङ्सन | एजेन्टका सबै फङ्सनहरू |
| **लचिलोपन** | परिभाषामा निश्चित | रनटाइममा गतिशील |
| **कन्टेक्स्ट** | सीमित | पूर्ण एजेन्ट कन्टेक्स्ट |
| **संयोजन** | धेरै डेकोरेटरहरू | मिडलवेयर पाइपलाइन |
| **एजेन्ट-जागरूक** | छैन | हो (एजेन्टको अवस्थामा पहुँच) |

### 📚 मिडलवेयर कहिले प्रयोग गर्ने:

✅ **मिडलवेयर प्रयोग गर्नुहोस् जब:**
- तपाईंले प्रयोगकर्ता/सत्रको अवस्थाको आधारमा व्यवहार परिमार्जन गर्न आवश्यक छ
- तपाईंले धेरै फङ्सनहरूमा तर्क लागू गर्न चाहनुहुन्छ
- तपाईंलाई एजेन्ट-स्तरको कन्टेक्स्टमा पहुँच आवश्यक छ
- तपाईंले क्रस-कटिङ चासोहरू (लगिङ, प्रमाणीकरण, आदि) कार्यान्वयन गर्दै हुनुहुन्छ

❌ **मिडलवेयर प्रयोग नगर्नुहोस् जब:**
- साधारण इनपुट मान्यता (Pydantic प्रयोग गर्नुहोस्)
- फङ्सन-विशिष्ट तर्क (फङ्सनमै राख्नुहोस्)
- एकपटकको परिमार्जन (फङ्सन परिवर्तन गर्नुहोस्)

### 🎓 उन्नत ढाँचाहरू:

```python
# Multiple middleware (execution order matters!)
middleware=[
    logging_middleware,      # Logs first
    auth_middleware,         # Then checks auth
    cache_middleware,        # Then checks cache
    rate_limit_middleware,   # Then rate limits
    priority_check_middleware  # Finally priority check
]

# Conditional middleware execution
async def conditional_middleware(context, next):
    if should_execute(context):
        await next(context)
        # Modify result
    else:
        # Skip execution entirely
        context.result = cached_value
```

### 🔗 सम्बन्धित अवधारणाहरू:

- **एजेन्ट मिडलवेयर**: agent.run() कलहरू रोक्छ
- **फङ्सन मिडलवेयर**: टूल फङ्सन कलहरू रोक्छ (हामीले प्रयोग गरेको!)
- **मिडलवेयर पाइपलाइन**: क्रम अनुसार कार्यान्वयन हुने मिडलवेयरको श्रृंखला
- **कन्टेक्स्ट प्रसारण**: मिडलवेयर श्रृंखलामा अवस्था पास गर्नुहोस्



---

**अस्वीकरण**:  
यो दस्तावेज़ AI अनुवाद सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) प्रयोग गरेर अनुवाद गरिएको हो। हामी यथासम्भव शुद्धता सुनिश्चित गर्न प्रयास गर्छौं, तर कृपया ध्यान दिनुहोस् कि स्वचालित अनुवादमा त्रुटिहरू वा अशुद्धताहरू हुन सक्छ। मूल दस्तावेज़ यसको मातृभाषामा आधिकारिक स्रोत मानिनुपर्छ। महत्वपूर्ण जानकारीको लागि, व्यावसायिक मानव अनुवाद सिफारिस गरिन्छ। यस अनुवादको प्रयोगबाट उत्पन्न हुने कुनै पनि गलतफहमी वा गलत व्याख्याको लागि हामी जिम्मेवार हुनेछैनौं।
